## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,kodiak,US,57.7900,-152.4072,36.91,93,100,8.05,mist
1,1,kapaa,US,22.0752,-159.3190,81.12,67,0,1.01,clear sky
2,2,butaritari,KI,3.0707,172.7902,81.79,78,100,16.22,light rain
3,3,nishihara,JP,26.1842,127.7558,69.08,88,75,21.85,broken clouds
4,4,buala,SB,-8.1450,159.5921,80.04,84,100,10.20,light rain
...,...,...,...,...,...,...,...,...,...,...
532,532,nanzhang,CN,31.7765,111.8386,39.20,44,99,0.85,overcast clouds
533,533,noumea,NC,-22.2763,166.4572,87.78,63,67,9.60,broken clouds
534,534,iralaya,HN,15.0000,-83.2333,76.55,80,1,6.46,clear sky
535,535,snasa,NO,64.2457,12.3778,10.67,95,100,5.57,overcast clouds


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.count()

City_ID                108
City                   108
Country                103
Lat                    108
Lng                    108
Max Temp               108
Humidity               108
Cloudiness             108
Wind Speed             108
Current Description    108
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_cities_df = filtered_cities_df.dropna()
filtered_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,nishihara,JP,26.1842,127.7558,69.08,88,75,21.85,broken clouds
15,15,santo antonio do ica,BR,-3.1022,-67.9397,73.81,97,100,2.64,overcast clouds
21,21,mar del plata,AR,-38.0023,-57.5575,71.62,83,75,5.75,light rain
24,24,ponta do sol,PT,32.6667,-17.1000,66.70,78,99,8.79,overcast clouds
27,27,esperance,AU,-33.8667,121.9000,69.64,53,6,15.50,clear sky
...,...,...,...,...,...,...,...,...,...,...
514,514,maun,BW,-19.9833,23.4167,70.07,79,9,5.75,clear sky
516,516,kaili,ID,-3.4626,120.2147,72.43,84,100,1.19,overcast clouds
517,517,santa cruz de tenerife,ES,28.4682,-16.2546,73.60,70,100,4.00,light rain
518,518,san carlos de bariloche,AR,-41.1456,-71.3082,69.55,46,40,10.36,scattered clouds


In [8]:
filtered_cities_df.count()

City_ID                103
City                   103
Country                103
Lat                    103
Lng                    103
Max Temp               103
Humidity               103
Cloudiness             103
Wind Speed             103
Current Description    103
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,nishihara,JP,69.08,broken clouds,26.1842,127.7558,
15,santo antonio do ica,BR,73.81,overcast clouds,-3.1022,-67.9397,
21,mar del plata,AR,71.62,light rain,-38.0023,-57.5575,
24,ponta do sol,PT,66.70,overcast clouds,32.6667,-17.1000,
27,esperance,AU,69.64,clear sky,-33.8667,121.9000,
34,upington,ZA,71.98,broken clouds,-28.4478,21.2561,
35,phalombe,MW,66.02,overcast clouds,-15.8064,35.6507,
39,cabo san lucas,MX,72.70,broken clouds,22.8909,-109.9124,
46,dickinson,US,70.20,mist,29.4608,-95.0513,
57,cape town,ZA,66.11,broken clouds,-33.9258,18.4232,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    #Add Lat and Long to key (location) for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,nishihara,JP,69.08,broken clouds,26.1842,127.7558,Minshuku Agaihama
15,santo antonio do ica,BR,73.81,overcast clouds,-3.1022,-67.9397,HOTEL AMAZON WAY
21,mar del plata,AR,71.62,light rain,-38.0023,-57.5575,Gran Hotel Mar del Plata
24,ponta do sol,PT,66.70,overcast clouds,32.6667,-17.1000,Hotel do Campo
27,esperance,AU,69.64,clear sky,-33.8667,121.9000,Hospitality Esperance


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   103
Country                103
Max Temp               103
Current Description    103
Lat                    103
Lng                    103
Hotel Name             103
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,nishihara,JP,69.08,broken clouds,26.1842,127.7558,Minshuku Agaihama
1,15,santo antonio do ica,BR,73.81,overcast clouds,-3.1022,-67.9397,HOTEL AMAZON WAY
2,21,mar del plata,AR,71.62,light rain,-38.0023,-57.5575,Gran Hotel Mar del Plata
3,24,ponta do sol,PT,66.70,overcast clouds,32.6667,-17.1000,Hotel do Campo
4,27,esperance,AU,69.64,clear sky,-33.8667,121.9000,Hospitality Esperance


In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))